In [17]:
from neo4j import GraphDatabase
import json

In [18]:
uri = "neo4j://127.0.0.1:7687"
username = "neo4j"
password = "88888888"

driver = GraphDatabase.driver(uri, auth=(username, password))

In [19]:
def create_constraints(tx):
    tx.run("CREATE CONSTRAINT IF NOT EXISTS FOR (p:Person) REQUIRE p.person_id IS UNIQUE")
    tx.run("CREATE CONSTRAINT IF NOT EXISTS FOR (d:Document) REQUIRE d.doc_id IS UNIQUE")

In [20]:
def insert_person_and_doc(tx, person_id, name, traits, doc_id, text):
    tx.run("""
        MERGE (p:Person {person_id: $person_id})
        SET p.name = $name,
            p.openness = $openness,
            p.conscientiousness = $conscientiousness,
            p.extraversion = $extraversion,
            p.agreeableness = $agreeableness,
            p.neuroticism = $neuroticism
        MERGE (d:Document {doc_id: $doc_id})
        SET d.text = $text
        MERGE (p)-[:HAS_DOCUMENT]->(d)
    """, 
    person_id=person_id, name=name, 
    openness=traits["openness"], conscientiousness=traits["conscientiousness"],
    extraversion=traits["extraversion"], agreeableness=traits["agreeableness"],
    neuroticism=traits["neuroticism"],
    doc_id=doc_id, text=text)

In [21]:
def load_and_write_to_neo4j(prediction_file):
    with open(prediction_file, "r") as f:
        data = json.load(f)
    
    with driver.session() as session:
        session.execute_write(create_constraints)
        for i, record in enumerate(data):
            person_id = f"person_{i+1}"
            name = f"Person_{i+1}"
            doc_id = f"doc_{i+1}"
            text = record["document"]
            traits = record["predicted_traits"]
            session.execute_write(insert_person_and_doc, person_id, name, traits, doc_id, text)
    
    print(f"✅ Imported {len(data)} records into Neo4j.")

In [23]:
load_and_write_to_neo4j("mlp_predictions.json")

✅ Imported 15 records into Neo4j.
